# Libraries needed

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.layers import Dense, Dropout
from keras.layers import LSTM
from keras.models import Sequential
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import GridSearchCV
from keras.models import model_from_json
from sklearn.metrics import confusion_matrix
from urad_Preprocessing import get_data_preprocessing
time_prediction = 7

Using TensorFlow backend.


Prepare the dataset

In [2]:
np.random.seed(5) #set pseudo checkpoint
dataset = pd.read_csv("fb_ads_preprocess.csv", header=None)
dataset = dataset.iloc[:,-1]
len_dataset = len(dataset)
dataset = dataset.values.reshape(len_dataset, 1)
print(dataset.shape)

(510, 1)


# Function List ("Data splitting (training and testing)", "ANN", "LSTM", "dataset converter")

In [3]:
# convert an array of values into a dataset matrix
def convert_cpc(dataset, time_prediction=7):
    # look back default value is in day, it is equal to one week
    dataX, dataY = [], []
    for i in range(len(dataset)-time_prediction+1):
        a = dataset[i:i+time_prediction-1]
        dataX.append(a)
        dataY.append(dataset[i + time_prediction-1])
    return np.array(dataX), np.array(dataY)

Function List ("Data splitting (training and testing)", "ANN", "LSTM", "dataset converter")

In [4]:
#Divide the data into training, and test data
def divide_data(dataset, time_prediction):
    datasetX, datasetY= convert_cpc(dataset, time_prediction)
    #prepare the dataset, and divide them into training and testing dataset
    train_size = int(len_dataset * 0.8)
    test_size = len_dataset - train_size
    trainX, trainY, testX, testY =  datasetX[:train_size], datasetY[:train_size], \
                                    datasetX[train_size:], datasetY[train_size:]
    trainX = trainX.reshape(-1, 1, trainX.shape[1])
    testX = testX.reshape(-1, 1, testX.shape[1])
    print (datasetX.shape, trainX.shape, testX.shape, len_dataset)
    return trainX, trainY, testX, testY

In [5]:
#Make the data into classification problem (up and down)
def substract(x1, x0):
    return (x1 - x0) > 0

def regression_to_class(inp):
    len_input = len(inp)
    output = np.zeros_like(inp)
    for i in range(1,len_input): output[i,0] = substract(inp[i,0], inp[i-1,0])
    return output 

In [7]:
#build LSTM model
batch_size = 128

# reshape input to be [samples, time steps, features]
trainX, trainY, testX, testY = divide_data(dataset, time_prediction)
def build_model_LSTM(optimizer_value, time_prediction):    
    print("LSTM", trainX.shape, trainY.shape, testX.shape, testY.shape)
    input_shape = np.array([1, time_prediction - 1])
    lstm_model = Sequential()
    lstm_model.add( LSTM(time_prediction*4, input_shape = input_shape) )
    lstm_model.add(Dropout(0.5))
    lstm_model.add(Dense(1))
    adam = Adam(decay = optimizer_value*0.1, lr=optimizer_value)
    lstm_model.compile(loss='mean_squared_error', optimizer = adam)
    return lstm_model

(504, 6, 1) (408, 1, 6) (96, 1, 6) 510


# Validation

Do Validation and See its Performance 

In [8]:
#using the training data that will be divided again into 80% train, and 20% validation
lstm_model = build_model_LSTM(0.001, time_prediction)
history = lstm_model.fit(trainX, trainY, epochs=1000, batch_size=batch_size, validation_split=0.2, shuffle=True)

LSTM (408, 1, 6) (408, 1) (96, 1, 6) (96, 1)
Train on 326 samples, validate on 82 samples
Epoch 1/1000


InternalError: Blas GEMM launch failed : a.shape=(128, 6), b.shape=(6, 28), m=128, n=28, k=6
	 [[Node: lstm_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/gpu:0"](lstm_1/Reshape, lstm_1/strided_slice)]]
	 [[Node: loss/mul/_51 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_2362_loss/mul", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'lstm_1/MatMul', defined at:
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Program Files\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Program Files\Anaconda3\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Program Files\Anaconda3\lib\site-packages\trait

In [ ]:
#plot training and validation loss using MSE 
print(history.history.keys())
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('ANN: model missing accuracy')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.ylim((.0,0.05))
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Prediction

Do The Prediction and Re-Evaluate The Model Performance

In [ ]:
lstm_model = build_model_LSTM(0.001, time_prediction)
lstm_model.fit(trainX, trainY, epochs=1000, batch_size=batch_size)
train_predict = lstm_model.predict(trainX)
test_predict = lstm_model.predict(testX)

In [ ]:
#Find the accuracy of our model
y_label = np.append(trainY, testY, axis = 0).reshape(-1,1)
train_predict_class, test_predict_class = regression_to_class(train_predict), regression_to_class(test_predict)
real_class = regression_to_class(y_label)
print (real_class.shape, train_predict_class.shape, test_predict_class.shape)

In [ ]:
print(train_predict.shape, real_class.shape)
print (train_predict_class[:5,0], real_class[:5,0])
# tn, fp, fn, tp = confusion_matrix(real_class[len(train_predict):-1,0], test_predict_class[1:,0]).ravel()
tn, fp, fn, tp = confusion_matrix(real_class[:len(train_predict)-1], train_predict_class[1:]).ravel()
print (tn, fp, fn, tp)
print ("Performance's Accuracy: ",(tn+tp)* 100/ (tn + fp + fn + tp))
print ("Sensitivity or True Positive Rate: ", tp*100 / (tp+fn) )
print ("Specificity or True Negative Rate: ", tn*100 / (tn+fp) )

In [ ]:
#----------plot the prediction result
# shift train predictions for plotting
time_prediction = 7
train_predict_plot = np.empty_like(dataset)
train_predict_plot[:, :] = np.nan
train_predict_plot[time_prediction-1:len(train_predict)+time_prediction-1, :] = train_predict

# shift test predictions for plotting
test_predict_plot = np.empty_like(dataset)
test_predict_plot[:, :] = np.nan
test_predict_plot[len(train_predict) + time_prediction-1:, :] = test_predict

# plot baseline and predictions
plot_dataset = np.zeros((len_dataset, 1))
plot_dataset[:,:] = np.nan
plot_dataset[time_prediction-1:] = y_label
plt.plot(plot_dataset)
plt.plot(train_predict_plot)
plt.plot(test_predict_plot)
plt.title('ANN: Prediction Results')
plt.ylabel('CPC')
plt.xlabel('Dataset')
plt.legend(['dataset', 'training accuracy', 'testing accuracy'], loc='upper left')
plt.show()

# Forecasting

Forecasting the CPC for n-days

In [ ]:
#LSTM
forecast, inp, predict = np.array([]), .0, .0
adding = 7
for i in range(time_prediction+adding):
    if i > 0: inp = np.concatenate([inp[-1, 0, 1:], predict[0] ]).reshape((1, 1,time_prediction-1) )
    else: inp = np.concatenate([trainX[-1, 0, 1:], train_predict[-1]]).reshape((1, 1, time_prediction-1) )
    predict = lstm_model.predict(inp)
    forecast = np.append(forecast, predict)

In [ ]:
#------------plot the forecasting result
plot_dataset = np.zeros((len_dataset, 1))
plot_dataset[:,:] = np.nan
plot_dataset[time_prediction-1:] = y_label
plt.plot(plot_dataset)

# shift test forecasting for plotting
forecast = forecast.reshape((time_prediction+adding,1))
forecast_plot = np.empty_like(dataset)
forecast_plot [:, :] = np.nan
forecast_plot [len(train_predict) + time_prediction:len(train_predict) + (2*time_prediction)+adding, :] = forecast
plt.plot(train_predict_plot)
plt.plot(forecast_plot)


plt.title('LSTM: Prediction Results')
plt.ylabel('CPC')
plt.xlabel('Dataset')
plt.legend(['dataset', 'training', 'forecasting'], loc='upper left')
plt.show()

In [ ]:
forecast_result = regression_to_class(forecast)
test_data = regression_to_class(testY)
print (np.append(forecast_result[:14], test_data[:14], axis =1))

# Save the trained NN-weights, and Load it to Predict the Input

reshape the dataset (to be used for forecasting using ANN)

Save the model's weights

In [ ]:
model_json = lstm_model.to_json() # for LSTM
with open("model.json", "w") as json_file:
    json_file.write(model_json)
    
# serialize weights to HDF5
model.save_weights("model.h5")
print("Saved model to disk")

Load and compile the model's weights

In [ ]:
# load json and create model
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# compile loaded model
optimizer_value = 0.0001
adam = Adam(decay = optimizer_value*0.05, lr=optimizer_value)
loaded_model.compile(loss="mean_squared_error", optimizer = adam)

In [ ]:
train_predict_loaded_model = model.predict(trainX)
test_predict_loaded_model = model.predict(testX)

In [ ]:
print(train_predict_loaded_model.shape, test_predict_loaded_model.shape)

# Fine-Tuning process

In [ ]:
#------------LSTM:fine tuning using gridSearch
model = KerasRegressor(build_fn=build_model_LSTM)
parameters = {'batch_size' : [128, 256, 512], 
                'nb_epoch' : [100, 1000, 10000 ],
                'optimizer_value': [0.01, 0.001, 0.0001]}#,
#                'hidden_layer' : [1, 3, 5]}
fine_tune = GridSearchCV(estimator=model, 
                         param_grid=parameters,
                         cv = 10)
fine_tune_result = fine_tune.fit(trainX, trainY)
best_params = fine_tune_result.best_params_
best_accuracy = fine_tune_result.best_score_

In [ ]:
#LSTM: print the loss and the best parameters
means = fine_tune_result.cv_results_['mean_test_score']
stds = fine_tune_result.cv_results_['std_test_score']
params = fine_tune_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))